In [ ]:
#INSTALAÇÃO DO PACOTE PYSPARK
!pip install pyspark

     |████████████████████████████████| 281.4 MB 34 kB/s 
     |████████████████████████████████| 198 kB 49.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=a4b957e5492af2d5da83272e87bffe5c60756cf1cbf0c839cff927de9fe82faa
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
# INSTALAÇÃO DO  GOOGLE CLOUD
!pip install gcsfs

     |████████████████████████████████| 134 kB 42.8 MB/s 
     |████████████████████████████████| 1.1 MB 50.2 MB/s 
     |████████████████████████████████| 144 kB 56.0 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
     |████████████████████████████████| 271 kB 11.1 MB/s 


In [ ]:
# Instalação dos pacotes
from pyspark.sql import SparkSession
from pyspark import SparkConf
import pyspark.sql.functions as F
import pandas as pd
from google.cloud import storage
import os
from pyspark.sql.types import *
from pyspark.sql.window import Window


In [ ]:
# ESTABELEÇO A MONTAGEM DO DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# VALIDO A CONEXÃO COM O 
serviceAccount = '/content/plasma-geode-339918-e47bf06e25cf.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount


In [ ]:
client = storage.Client()

#CRIAR UMA VARIÁVEL CHAMADA BUCKET QUE VAI RECEBER O NOME DA BUCKET DO CLOUD STORAGE
bucket = client.get_bucket('individualproject')
#USAR O MÉTODO BLOB PARA RETORNAR O NOME DO ARQUIVO (JSON, CSV, PARQUET)
bucket.blob('marketing_campaign.csv')
#CRIA UMA VARIÁVEL PATH PARA COLOCAR O CAMINHO DO CSV
path = 'gs://individualproject/Original/marketing_campaign.csv'
df_pandas = pd.read_csv(path, sep=',')

In [ ]:
# ESTABELECE A CONEXÃO ENTRE SPAYPARK COM COLLAB (4050) /GCP 
spark = (
    SparkSession.builder
                .master('local')
                .appName('cloudStorage')
                .config('spark.ui.port', '4050')
                .config("spark.jars", 'https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop2-latest.jar')
                .getOrCreate()
)

In [ ]:
# CONEXÃO OK COM SPARK
spark

In [ ]:
# CHAMEI DIRETO DA GCP O ARQUIVO .CSV SEM TRANSFORMAR EM PANDAS.
df = ( spark.read
            .format("csv")
            .option("header", "true")
            .option("inferschema", "true")
            .option("delimiter", ",")
            .load(path)
      
)
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Year_Birth: integer (nullable = true)
 |-- Education: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Kidhome: integer (nullable = true)
 |-- Teenhome: integer (nullable = true)
 |-- Dt_Customer: string (nullable = true)
 |-- Recency: integer (nullable = true)
 |-- MntWines: integer (nullable = true)
 |-- MntFruits: integer (nullable = true)
 |-- MntMeatProducts: integer (nullable = true)
 |-- MntFishProducts: integer (nullable = true)
 |-- MntSweetProducts: integer (nullable = true)
 |-- MntGoldProds: integer (nullable = true)
 |-- NumDealsPurchases: integer (nullable = true)
 |-- NumWebPurchases: integer (nullable = true)
 |-- NumCatalogPurchases: integer (nullable = true)
 |-- NumStorePurchases: integer (nullable = true)
 |-- NumWebVisitsMonth: integer (nullable = true)
 |-- AcceptedCmp3: integer (nullable = true)
 |-- AcceptedCmp4: integer (nullable = true)
 |-- AcceptedC

In [ ]:
#CRIANDO SCHEMAS UTILIZANDO STRUCT TYPE
schema = StructType([
                 StructField('ID', IntegerType(), True),   
                 StructField('Year_Birth', IntegerType(), True),
                 StructField('Education', StringType(), True),
                 StructField('Marital_Status', StringType(), True),
                 StructField('Income', DoubleType(), True),
                 StructField('Kidhome', IntegerType(), True),
                 StructField('Qtd_adolescente', IntegerType(), True),
                 StructField('Teenhome', IntegerType(), True),
                 StructField('Dt_Customer', StringType(), True),
                 StructField('Recency', IntegerType(), True),
                 StructField('MntWines', IntegerType(), True),
                 StructField('MntFruits', IntegerType(), True),
                 StructField('MntMeatProducts', IntegerType(), True),
                 StructField('MntFishProducts', IntegerType(), True),
                 StructField('MntSweetProducts', IntegerType(), True),
                 StructField('MntGoldProds', IntegerType(), True),
                 StructField('NumDealsPurchases', IntegerType(), True),
                 StructField('NumWebPurchases', IntegerType(), True),
                 StructField('NumCatalogPurchases', IntegerType(), True),
                 StructField('NumStorePurchases', IntegerType(), True),
                 StructField('NumWebVisitsMonth', StringType(), True),
                 StructField('AcceptedCmp3', IntegerType(), True),
                 StructField('AcceptedCmp4', IntegerType(), True),
                 StructField('AcceptedCmp5', IntegerType(), True),
                 StructField('AcceptedCmp1', IntegerType(), True),
                 StructField('AcceptedCmp2', IntegerType(), True),
                 StructField('Complain', IntegerType(), True),
                 StructField('Z_CostContact', IntegerType(), True),
                 StructField('Z_Revenue', IntegerType(), True),
                 StructField('Response', IntegerType(), True),

]   
)


In [ ]:
#RETORNA O SCHEMA DO DATAFRAME
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Year_Birth: integer (nullable = true)
 |-- Education: string (nullable = true)
 |-- Marital_Status: string (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Kidhome: integer (nullable = true)
 |-- Teenhome: integer (nullable = true)
 |-- Dt_Customer: string (nullable = true)
 |-- Recency: integer (nullable = true)
 |-- MntWines: integer (nullable = true)
 |-- MntFruits: integer (nullable = true)
 |-- MntMeatProducts: integer (nullable = true)
 |-- MntFishProducts: integer (nullable = true)
 |-- MntSweetProducts: integer (nullable = true)
 |-- MntGoldProds: integer (nullable = true)
 |-- NumDealsPurchases: integer (nullable = true)
 |-- NumWebPurchases: integer (nullable = true)
 |-- NumCatalogPurchases: integer (nullable = true)
 |-- NumStorePurchases: integer (nullable = true)
 |-- NumWebVisitsMonth: integer (nullable = true)
 |-- AcceptedCmp3: integer (nullable = true)
 |-- AcceptedCmp4: integer (nullable = true)
 |-- AcceptedC

In [ ]:
#EXIBINDO ALGUMAS COLUNAS
df.show(5)

+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+------------+------------+------------+------------+------------+--------+-------------+---------+--------+
|  ID|Year_Birth| Education|Marital_Status|Income|Kidhome|Teenhome|Dt_Customer|Recency|MntWines|MntFruits|MntMeatProducts|MntFishProducts|MntSweetProducts|MntGoldProds|NumDealsPurchases|NumWebPurchases|NumCatalogPurchases|NumStorePurchases|NumWebVisitsMonth|AcceptedCmp3|AcceptedCmp4|AcceptedCmp5|AcceptedCmp1|AcceptedCmp2|Complain|Z_CostContact|Z_Revenue|Response|
+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+----------

In [ ]:
# EXIBO O QUE FOI IMPORTADO
df.columns

['ID',
 'Year_Birth',
 'Education',
 'Marital_Status',
 'Income',
 'Kidhome',
 'Teenhome',
 'Dt_Customer',
 'Recency',
 'MntWines',
 'MntFruits',
 'MntMeatProducts',
 'MntFishProducts',
 'MntSweetProducts',
 'MntGoldProds',
 'NumDealsPurchases',
 'NumWebPurchases',
 'NumCatalogPurchases',
 'NumStorePurchases',
 'NumWebVisitsMonth',
 'AcceptedCmp3',
 'AcceptedCmp4',
 'AcceptedCmp5',
 'AcceptedCmp1',
 'AcceptedCmp2',
 'Complain',
 'Z_CostContact',
 'Z_Revenue',
 'Response']

In [ ]:
#MANTENHO UM BACKUP ANTES DA ELIMINAÇÃO
df_backup = df

In [ ]:
# VERIFIQUI QUAIS COLUNAS QUERO ELIMINAR QUE NÃO SERVIRÃO PARA A MINHA ANÁLISE
df.drop('Recency','AcceptedCmp4','AcceptedCmp5','AcceptedCmp1','AcceptedCmp2','Complain','Z_CostContact','Z_Revenue','Response').show()

+----+----------+----------+--------------+------+-------+--------+-----------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+------------+
|  ID|Year_Birth| Education|Marital_Status|Income|Kidhome|Teenhome|Dt_Customer|MntWines|MntFruits|MntMeatProducts|MntFishProducts|MntSweetProducts|MntGoldProds|NumDealsPurchases|NumWebPurchases|NumCatalogPurchases|NumStorePurchases|NumWebVisitsMonth|AcceptedCmp3|
+----+----------+----------+--------------+------+-------+--------+-----------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+------------+
|5524|      1957|Graduation|        Single| 58138|      0|       0| 04-09-2012|     635|       88|            546|            172|              88|          88|                3|              8|              

In [ ]:
df.show()

+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+------------+------------+------------+------------+------------+--------+-------------+---------+--------+
|  ID|Year_Birth| Education|Marital_Status|Income|Kidhome|Teenhome|Dt_Customer|Recency|MntWines|MntFruits|MntMeatProducts|MntFishProducts|MntSweetProducts|MntGoldProds|NumDealsPurchases|NumWebPurchases|NumCatalogPurchases|NumStorePurchases|NumWebVisitsMonth|AcceptedCmp3|AcceptedCmp4|AcceptedCmp5|AcceptedCmp1|AcceptedCmp2|Complain|Z_CostContact|Z_Revenue|Response|
+----+----------+----------+--------------+------+-------+--------+-----------+-------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+----------

In [ ]:
# EFETIVO A MINHA ELIMINAÇÃO
df = df.drop('Recency','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','AcceptedCmp1','AcceptedCmp2','Complain','Z_CostContact','Z_Revenue','Response')

In [ ]:
#EXIBO A MINHA ELIMINAÇÃO
df.show()

+----+----------+----------+--------------+------+-------+--------+-----------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+
|  ID|Year_Birth| Education|Marital_Status|Income|Kidhome|Teenhome|Dt_Customer|MntWines|MntFruits|MntMeatProducts|MntFishProducts|MntSweetProducts|MntGoldProds|NumDealsPurchases|NumWebPurchases|NumCatalogPurchases|NumStorePurchases|NumWebVisitsMonth|
+----+----------+----------+--------------+------+-------+--------+-----------+--------+---------+---------------+---------------+----------------+------------+-----------------+---------------+-------------------+-----------------+-----------------+
|5524|      1957|Graduation|        Single| 58138|      0|       0| 04-09-2012|     635|       88|            546|            172|              88|          88|                3|              8|                 10|                4|               

Renomenando as colunas para o idioma PT  
Os dados da planilha já foram tratados em Pandas. Optei em converter apenas os nomes das colunas

In [ ]:
df = df.withColumnRenamed('ID','Id_cod')\
.withColumnRenamed('Year_Birth','Ano_nascimento')\
.withColumnRenamed('Education','Escolaridade')\
.withColumnRenamed('Marital_Status','Estado_Civil')\
.withColumnRenamed('Income','Renda')\
.withColumnRenamed('Kidhome','Qtd_criancas')\
.withColumnRenamed('Teenhome','Qtd_Jovens')\
.withColumnRenamed('Dt_Customer','Data_Cliente')\
.withColumnRenamed('MntWines','Gastos_vinhos')\
.withColumnRenamed('MntFruits','Gastos_Futas')\
.withColumnRenamed('MntMeatProducts','Gastos_Carne')\
.withColumnRenamed('MntFishProducts','Gastos_Pescados')\
.withColumnRenamed('MntSweetProducts','Gastos_Doces')\
.withColumnRenamed('MntGoldProds','Gastos_Caros')\
.withColumnRenamed('NumDealsPurchases','Ofertas_compras')\
.withColumnRenamed('NumWebPurchases','Compras_Internet')\
.withColumnRenamed('NumCatalogPurchases','Compra_Catálogo')\
.withColumnRenamed('NumStorePurchases','Compra_Loja')\
.withColumnRenamed('NumWebVisitsMonth','Visitas_Site')\


In [ ]:
df.show()

+------+--------------+------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+
|Id_cod|Ano_nascimento|Escolaridade|Estado_Civil|Renda|Qtd_criancas|Qtd_Jovens|Data_Cliente|Gastos_vinhos|Gastos_Futas|Gastos_Carne|Gastos_Pescados|Gastos_Doces|Gastos_Caros|Ofertas_compras|Compras_Internet|Compra_Catálogo|Compra_Loja|Visitas_Site|
+------+--------------+------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+
|  5524|          1957|  Graduation|      Single|58138|           0|         0|  04-09-2012|          635|          88|         546|            172|          88|          88|              3|               8|             10|          4|           7|
|  2

Renomear linhas

In [ ]:
# Renomenando as linhas para o idioma PT
df=df.replace(['Graduation','PhD','Master','2n Cycle','Basic'],['Graduação','PhD','Mestrado','Segundo Ciclo','Básico'])
df=df.replace(['Single','Married','Divorced','Together'],['Solteiro','Casado','Divorciado','Uni_Estavel'])


In [ ]:
df.show()

+------+--------------+-------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+
|Id_cod|Ano_nascimento| Escolaridade|Estado_Civil|Renda|Qtd_criancas|Qtd_Jovens|Data_Cliente|Gastos_vinhos|Gastos_Futas|Gastos_Carne|Gastos_Pescados|Gastos_Doces|Gastos_Caros|Ofertas_compras|Compras_Internet|Compra_Catálogo|Compra_Loja|Visitas_Site|
+------+--------------+-------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+
|  5524|          1957|    Graduação|    Solteiro|58138|           0|         0|  04-09-2012|          635|          88|         546|            172|          88|          88|              3|               8|             10|          4|           7|


UTILIZAÇÃO DE FILTROS

In [ ]:
# FILTRANDO  SOMENTE POR UM TIPO DE ESCOLARIDADE (FILTER)
df.filter(df.Escolaridade == "Graduação").show(10)

+------+--------------+------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+
|Id_cod|Ano_nascimento|Escolaridade|Estado_Civil|Renda|Qtd_criancas|Qtd_Jovens|Data_Cliente|Gastos_vinhos|Gastos_Futas|Gastos_Carne|Gastos_Pescados|Gastos_Doces|Gastos_Caros|Ofertas_compras|Compras_Internet|Compra_Catálogo|Compra_Loja|Visitas_Site|
+------+--------------+------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+
|  5524|          1957|   Graduação|    Solteiro|58138|           0|         0|  04-09-2012|          635|          88|         546|            172|          88|          88|              3|               8|             10|          4|           7|
|  2

In [ ]:
# FILTRANDO  SOMENTE OS QUE TIVERAM GASTOS DE VINHOS > 99 DOS 50 PRIMEIROS
df.filter(df['Gastos_vinhos']>=99).show()

+------+--------------+------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+
|Id_cod|Ano_nascimento|Escolaridade|Estado_Civil|Renda|Qtd_criancas|Qtd_Jovens|Data_Cliente|Gastos_vinhos|Gastos_Futas|Gastos_Carne|Gastos_Pescados|Gastos_Doces|Gastos_Caros|Ofertas_compras|Compras_Internet|Compra_Catálogo|Compra_Loja|Visitas_Site|
+------+--------------+------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+
|  5524|          1957|   Graduação|    Solteiro|58138|           0|         0|  04-09-2012|          635|          88|         546|            172|          88|          88|              3|               8|             10|          4|           7|
|  4

In [ ]:
# OUTRA FORMA DE FAZER UM FILTRO USANDO UM TIPO DE ESCOLARIDADE (WHERE)
df.where(F.col('Escolaridade') == 'Mestrado').show()

+------+--------------+------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+
|Id_cod|Ano_nascimento|Escolaridade|Estado_Civil|Renda|Qtd_criancas|Qtd_Jovens|Data_Cliente|Gastos_vinhos|Gastos_Futas|Gastos_Carne|Gastos_Pescados|Gastos_Doces|Gastos_Caros|Ofertas_compras|Compras_Internet|Compra_Catálogo|Compra_Loja|Visitas_Site|
+------+--------------+------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+
|  7446|          1967|    Mestrado| Uni_Estavel|62513|           0|         1|  09-09-2013|          520|          42|          98|              0|          42|          14|              2|               6|              4|         10|           6|
|  8

ORDENAÇÃO DE COLUNA

In [ ]:
# PELO GRAU DE ESCOLARIDADE EXIBE QUEM TEVE MAIS CONSUMO DE CARNES E VINHOS
df.select(F.col("Id_cod"),F.col("Escolaridade"), F.col("Gastos_vinhos"),F.col("Gastos_Carne")).orderBy(F.col("Gastos_Pescados").asc(), F.col("Gastos_vinhos")).show(5)


+------+-------------+-------------+------------+
|Id_cod| Escolaridade|Gastos_vinhos|Gastos_Carne|
+------+-------------+-------------+------------+
| 10401|Segundo Ciclo|            1|           5|
|  3068|    Graduação|            1|           9|
|  8234|       Básico|            1|           7|
|  6961|    Graduação|            1|           5|
| 11003|Segundo Ciclo|            2|           7|
+------+-------------+-------------+------------+
only showing top 5 rows



GROUPBY

In [ ]:
# AGRUPO POR TIPO DE ESCOLARIDADE E GASTO COM VINHOS 
df.groupBy("Escolaridade").sum('Gastos_vinhos').show()


+-------------+------------------+
| Escolaridade|sum(Gastos_vinhos)|
+-------------+------------------+
|Segundo Ciclo|             40231|
|          PhD|            196585|
|     Mestrado|            123238|
|       Básico|               391|
|    Graduação|            320371|
+-------------+------------------+



In [ ]:
# QUAL O GRAU DE ESCOLARIDADE QUE TEM CRIANÇAS
df.groupBy("Escolaridade").sum('Qtd_criancas','Qtd_Jovens').show()

+-------------+-----------------+---------------+
| Escolaridade|sum(Qtd_criancas)|sum(Qtd_Jovens)|
+-------------+-----------------+---------------+
|Segundo Ciclo|               97|             83|
|          PhD|              195|            291|
|     Mestrado|              168|            198|
|       Básico|               34|              5|
|    Graduação|              501|            557|
+-------------+-----------------+---------------+



In [ ]:
# QUAL O ESTADO CIVIL  QUE TEM CRIANÇAS
df.groupBy("Estado_Civil").sum('Qtd_criancas','Qtd_Jovens').show()

+------------+-----------------+---------------+
|Estado_Civil|sum(Qtd_criancas)|sum(Qtd_Jovens)|
+------------+-----------------+---------------+
|        YOLO|                0|              2|
| Uni_Estavel|              261|            307|
|  Divorciado|               96|            137|
|      Absurd|                0|              0|
|    Solteiro|              223|            195|
|       Widow|               18|             49|
|      Casado|              394|            442|
|       Alone|                3|              2|
+------------+-----------------+---------------+



In [ ]:
# QUAl O NIVEL DE ESCOLARIDADE NA PLANILHA ??
df.groupBy("Escolaridade").count().show()

+-------------+-----+
| Escolaridade|count|
+-------------+-----+
|Segundo Ciclo|  203|
|          PhD|  486|
|     Mestrado|  370|
|       Básico|   54|
|    Graduação| 1127|
+-------------+-----+



In [ ]:
# QUAL ESCOLARIDADE O CONSUMO DE CARNE FOI MAIOR ?? 
df.groupBy("Escolaridade").max('Gastos_Carne').show()

+-------------+-----------------+
| Escolaridade|max(Gastos_Carne)|
+-------------+-----------------+
|Segundo Ciclo|             1607|
|          PhD|             1622|
|     Mestrado|              925|
|       Básico|              122|
|    Graduação|             1725|
+-------------+-----------------+



WINDOW FUNCTION

from pyspark.sql.window import Window (INSTALADO)

Agregação
- Dados da pesquisa anterior

 - QUAL ESCOLARIDADE O CONSUMO DE CARNE FOI MAIOR ??
 - QUAl O NIVEL DE ESCOLARIDADE NA PLANILHA ??

In [ ]:
# AGREGACAO DE COLUNAS 
schema = ['Escolaridade','count','Gastos_Carne','Qtd_criancas','Qtd_Jovens']
dados = [('2n Cycle', 203, 1607,97,83,),('PhD',486,1622,195,291),('Master',370,925,168,198),('Graduation', 1127, 1725,501,557),(' Basic',54, 122,34,5)]
df2 = spark.createDataFrame(data=dados, schema=schema)


In [ ]:
df2.show()

+------------+-----+------------+------------+----------+
|Escolaridade|count|Gastos_Carne|Qtd_criancas|Qtd_Jovens|
+------------+-----+------------+------------+----------+
|    2n Cycle|  203|        1607|          97|        83|
|         PhD|  486|        1622|         195|       291|
|      Master|  370|         925|         168|       198|
|  Graduation| 1127|        1725|         501|       557|
|       Basic|   54|         122|          34|         5|
+------------+-----+------------+------------+----------+



In [ ]:
#Particionar minha janela a partir de uma coluna
w0 = Window.partitionBy(F.col('Estado_Civil')).orderBy('Renda')

In [ ]:
#row_number
df.withColumn('row_number', F.row_number().over(w0)).show(5)

+------+--------------+------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+----------+
|Id_cod|Ano_nascimento|Escolaridade|Estado_Civil|Renda|Qtd_criancas|Qtd_Jovens|Data_Cliente|Gastos_vinhos|Gastos_Futas|Gastos_Carne|Gastos_Pescados|Gastos_Doces|Gastos_Caros|Ofertas_compras|Compras_Internet|Compra_Catálogo|Compra_Loja|Visitas_Site|row_number|
+------+--------------+------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+----------+
|  4369|          1957|    Mestrado|      Absurd|65487|           0|         0|  10-01-2014|          240|          67|         500|            199|           0|         163|              3|               3|             

CRIANDO UMA NOVA COLUNA COM BASE EM DUAS CONDIÇÕES

In [ ]:
#Adicionando colunas df_spaypark
df.withColumn('Estado_Civil_novo',df ['Estado_Civil']+1).show ()

+------+--------------+-------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+-----------------+
|Id_cod|Ano_nascimento| Escolaridade|Estado_Civil|Renda|Qtd_criancas|Qtd_Jovens|Data_Cliente|Gastos_vinhos|Gastos_Futas|Gastos_Carne|Gastos_Pescados|Gastos_Doces|Gastos_Caros|Ofertas_compras|Compras_Internet|Compra_Catálogo|Compra_Loja|Visitas_Site|Estado_Civil_novo|
+------+--------------+-------------+------------+-----+------------+----------+------------+-------------+------------+------------+---------------+------------+------------+---------------+----------------+---------------+-----------+------------+-----------------+
|  5524|          1957|    Graduação|    Solteiro|58138|           0|         0|  04-09-2012|          635|          88|         546|            172|          88|          88|              3|     

In [ ]:
# APÓS O TÉRMINO DO TRABALHO , DEVEMOS SUBIR ESSE ARQUIVO .CSV PARA A GCP ( GOOGLE CLOUD PLATAFORM)
(df.write.format('csv')
.option('header', 'true')
.option('inferschema', 'true')
.option('delimiter', ';')
.save('gs://individualproject/Final/marketing_campaign_final.csv')
)
